# Tutorial: Import and export

<figure style="display: table; text-align:center; margin-left: auto; margin-right:auto">
    
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/simphony/docs/master?filepath=docs%2Fsource%2Fjupyter%2Fimport_export.ipynb \"Click to run the tutorial yourself!\")
    
</figure>

In this tutorial we will be covering the import and export capabilities of OSP-core. The utility functions that provide these functionalities are `import_cuds` and `export_cuds`, respectively.

<div class="admonition important">
<div class="admonition-title" style="font-weight: bold"><div style="display: inline-block">Tip</div></div>
The full API specifictions of the import and export functions can be found in the
[utilities API reference page](../api_ref.html#osp.core.utils.export_cuds).
</div>
</div>


For our running example, we'll be using the *city ontology* that was already introduces in the [cuds API tutorial](./cuds_api.html). First, let's create a few CUDS objects:

In [1]:
from osp.core.namespaces import city

c = city.City(name="Freiburg", coordinates=[47, 7])
p1 = city.Citizen(name="Peter")
p2 = city.Citizen(name="Anne")
c.add(p1, rel=city.hasInhabitant)
c.add(p2, rel=city.hasInhabitant)

<city.Citizen: 2b5d0a3f-81a5-4746-aab9-40adcb65e71f,  CoreSession: @0x261546f0eb0>

Now we can use the `export_cuds` methods to export the data into a file:

In [2]:
from osp.core.utils import export_cuds

export_cuds(c, file='./data.ttl', format='turtle')

This will create the file `data.ttl` with the following content:

In [3]:
from sys import platform

if platform == 'win32':
    !more data.ttl
else:
    !cat data.ttl

@prefix city: <http://www.osp-core.com/city#> .
@prefix cuba: <http://www.osp-core.com/cuba#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

cuba:_serialization rdf:first "d886f8ce-1326-40f5-a98b-c4c893b8c085" .

<http://www.osp-core.com/cuds#2b5d0a3f-81a5-4746-aab9-40adcb65e71f> a city:Citizen ;
    city:INVERSE_OF_hasInhabitant <http://www.osp-core.com/cuds#d886f8ce-1326-40f5-a98b-c4c893b8c085> ;
    city:age 25 ;
    city:name "Anne" .

<http://www.osp-core.com/cuds#766b320a-7e9a-43ec-a696-96b4f9ee494d> a city:Citizen ;
    city:INVERSE_OF_hasInhabitant <http://www.osp-core.com/cuds#d886f8ce-1326-40f5-a98b-c4c893b8c085> ;
    city:age 25 ;
    city:name "Peter" .

<http://www.osp-core.com/cuds#d886f8ce-1326-40f5-a98b-c4c893b8c085> a city:City ;
    city:coordinates "[47, 7]"^^<http://www.osp-core.com/cuba#_datatypes/VECTOR-INT-2> ;
    city:hasInhabitant <http://www.osp-core.com/cuds#2b5d0a3f-81a5-4746-aab9-40adcb65

You can change the format by entering a different value for the parameter `format`. The supported formats are “xml”, “n3”, “turtle”, “nt”, “pretty-xml”, “trix”, “trig” and “nquads”.

To import data, we can use the `import` method. Let's assume we wish to import data into an SQLite session. The following code will help us to achieve our aim:

In [4]:
from osp.wrappers.sqlite import SqliteSession
from osp.core.utils import import_cuds

with SqliteSession("test.db") as session:
    wrapper = city.CityWrapper(session=session)
    c = import_cuds('./data.ttl')
    wrapper.add(c)
    session.commit()

Now we can verify the data was indeed imported:

In [5]:
from osp.core.utils import pretty_print

with SqliteSession("test.db") as session:
    wrapper = city.CityWrapper(session=session)
    pretty_print(wrapper) 

- Cuds object:
  uid: 064d9794-a12d-4331-9821-44f0538ed48a
  type: city.CityWrapper
  superclasses: city.CityWrapper, cuba.Entity, cuba.Wrapper
  description: 
    To Be Determined

   |_Relationship city.hasPart:
     -  city.City cuds object named <Freiburg>:
     .  uid: 72595bc4-1b68-46a3-97e9-8f3de2650f2c
     .  coordinates: [47  7]
     .   |_Relationship city.hasInhabitant:
     .     -  city.Citizen cuds object named <Anne>:
     .     .  uid: 92a00459-0927-438c-a305-a26512ac7f03
     .     .  age: 25
     .     -  city.Citizen cuds object named <Peter>:
     .        uid: 27d1e83b-4ee9-4f4f-adb5-0b01a3cc2c1b
     .        age: 25
     -  city.City cuds object named <Freiburg>:
        uid: d886f8ce-1326-40f5-a98b-c4c893b8c085
        coordinates: [47  7]
         |_Relationship city.hasInhabitant:
           -  city.Citizen cuds object named <Anne>:
           .  uid: 2b5d0a3f-81a5-4746-aab9-40adcb65e71f
           .  age: 25
           -  city.Citizen cuds object named <Pete

<div class="admonition important">
<div class="admonition-title" style="font-weight: bold"><div style="display: inline-block">Notes</div></div>
    
1. The format is automatically inferred from the file extension. To specify it explicitly, you can add the `format` parameter, like so: `import_cuds('./data.ttl', format='turtle')`.
1. The `session` parameter is optional and inferred automatically from the context that created by the `with` statement (see the  [tutorial on multiple wrappers](./multiple_wrappers.html) for more information). You can specify the session explicitly like so: `import_cuds('./data.ttl', session=session)`.
</div>